In [2]:
client_id = '3fc718227bd34b34beb8b6aac2dbe73e'
client_secret = 'b01009306e054e6ca87406d04d2ec916'

https://github.com/codingforentrepreneurs/30-Days-of-Python/commit/9c8f45e840483133c340f10583ad6cb3f5e03fb6

https://developer.spotify.com/dashboard/login

In [3]:
import base64
import datetime
from urllib.parse import urlencode

import requests


# In[14]:


class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
data = spotify.search("Big Life", search_type = 'track')

q=Big+Life&type=track


In [6]:
data

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Big+Life&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3zorWCDx017sz4UYP2fC9w'},
       'href': 'https://api.spotify.com/v1/artists/3zorWCDx017sz4UYP2fC9w',
       'id': '3zorWCDx017sz4UYP2fC9w',
       'name': 'Chronic Law',
       'type': 'artist',
       'uri': 'spotify:artist:3zorWCDx017sz4UYP2fC9w'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JO',
      'JP',
      

In [7]:
tracks = ['Fernando', 'Waterloo', 'Hello']

In [8]:
for i in data['tracks']['items']:
    print(i['artists'][0]['name'])
        #print(i['album']['release_date'])
        
    

    #print(data['tracks']['items'][0]['album']['release_date'])

Chronic Law
USS (Ubiquitous Synergy Seeker)
Thomas Bergersen
Night Ranger
Freeman
Giza
HDX RECORDS
Big Sean
Big Sean
Big Time Rush
Lil Uzi Vert
Lil Uzi Vert
The Notorious B.I.G.
The Notorious B.I.G.
JAY-Z
Thomas Rhett
Thomas Rhett
Thomas Rhett
Lil Skies
The Notorious B.I.G.


In [9]:
import pandas as pd

In [22]:
df3 = pd.read_csv("../songtexte_bereinigt_gekuerzt.csv")
df3

,Unnamed: 0,artist,song,text,number_of_tokens
0,0,ABBA,Ahe's My Kind Of Girl,look at her face it's a wonderful face and i...,741
1,1,ABBA,"Andante, Andante",take it easy with me please touch me gently ...,1381
2,2,ABBA,As Good As New,i'll never know why i had to go why i had to ...,1438
3,3,ABBA,Bang,making somebody happy is a question of give an...,1207
4,4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...,1223
...,...,...,...,...,...
50827,57642,Ziggy Marley,Friend,i wanna thank you for the things you've done...,691
50828,57643,Ziggy Marley,G7,seven richest countries in the world them hav...,1443
50829,57644,Ziggy Marley,Generation,many generation have passed away fighting for...,1391
50830,57645,Ziggy Marley,Good Old Days,irie days come on play let the angels fly let...,858


In [23]:
df3 = df3.set_index('artist')
df3.drop("Unnamed: 0", axis=1, inplace=True)
df3

,song,text,number_of_tokens
artist,,,
ABBA,Ahe's My Kind Of Girl,look at her face it's a wonderful face and i...,741
ABBA,"Andante, Andante",take it easy with me please touch me gently ...,1381
ABBA,As Good As New,i'll never know why i had to go why i had to ...,1438
ABBA,Bang,making somebody happy is a question of give an...,1207
ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...,1223
...,...,...,...
Ziggy Marley,Friend,i wanna thank you for the things you've done...,691
Ziggy Marley,G7,seven richest countries in the world them hav...,1443
Ziggy Marley,Generation,many generation have passed away fighting for...,1391


In [24]:
series1 = df3.song

In [25]:
series1

artist
ABBA            Ahe's My Kind Of Girl
ABBA                 Andante, Andante
ABBA                   As Good As New
ABBA                             Bang
ABBA                 Bang-A-Boomerang
                        ...          
Ziggy Marley                   Friend
Ziggy Marley                       G7
Ziggy Marley               Generation
Ziggy Marley            Good Old Days
Ziggy Marley            Hand To Mouth
Name: song, Length: 50832, dtype: object

In [26]:
artist = []
dates = []
title = []
series = pd.Series(dtype = str)
counter = 0
for k, v in series1.items():
    data = spotify.search(f"{v}", search_type = 'track')
    try:
        for i in data['tracks']['items']:
            if i['artists'][0]['name'] == f"{k}":
                tag = i['album']['release_date']
                dates.append(tag)
                artist.append(k)
                title.append(v)
        counter +=1
    except Exception:
        pass
# df = pd.DataFrame({'title': title, 'date': dates, 'artist': artist})
        


q=Ahe%27s+My+Kind+Of+Girl&type=track
q=Andante%2C+Andante&type=track
q=As+Good+As+New&type=track
q=Bang&type=track
q=Bang-A-Boomerang&type=track
q=Burning+My+Bridges&type=track
q=Cassandra&type=track
q=Chiquitita&type=track
q=Crazy+World&type=track
q=Crying+Over+You&type=track
q=Dance&type=track
q=Dancing+Queen&type=track
q=Disillusion&type=track
q=Does+Your+Mother+Know&type=track
q=Dream+World&type=track
q=Dum+Dum+Diddle&type=track
q=Eagle&type=track
q=Every+Good+Man&type=track
q=Fernando&type=track
q=Fernando+%28In+Spanish%29&type=track
q=Free+As+A+Bumble+Bee&type=track
q=From+A+Twinkling+Star+To+A+Passing+Angel&type=track
q=Gimme+Gimme+Gimme&type=track
q=Givin%27+A+Little+Bit+More&type=track
q=Gonna+Sing+You+My+Lovesong&type=track
q=Hamlet+III&type=track
q=Happy+Hawaii&type=track
q=Happy+New+Year&type=track
q=He+Is+Your+Brother&type=track
q=Head+Over+Heels&type=track
q=Here+We%27ll+Stay&type=track
q=Hey+Hey+Helen&type=track
q=Hole+In+Your+Soul&type=track
q=Honey%2C+Honey&type=track


q=I+Want+To+Give+It+All&type=track
q=I+Want+You&type=track
q=If+You+Love+Me&type=track
q=I%27ll+Be+Thinking+Of+You&type=track
q=I%27ll+Find+You&type=track
q=I%27ll+Never+Get+Enough+Of+You&type=track
q=I%27m+Alive&type=track
q=It%27s+Automatic&type=track
q=It%27s+Never+Too+Late&type=track
q=It%27s+Not+Easy&type=track
q=It%27s+Not+Too+Late&type=track
q=I%27ve+Got+Your+Love&type=track
q=Just+Another+Woman&type=track
q=Just+As+I+Am&type=track
q=Just+Between+The+Lines&type=track
q=Late+Again&type=track
q=Love+Comes+To+Me&type=track
q=Love+Conquers+Time&type=track
q=Love+Is+All&type=track
q=Love+Is+The+Arrow&type=track
q=Make+It+Right&type=track
q=Making+Love+Out+Of+Nothing+At+All&type=track
q=Miracles&type=track
q=More+Than+Natural&type=track
q=Mother+Said&type=track
q=My+Best+Friend&type=track
q=My+Hearts+With+You&type=track
q=Never+Fade+Away&type=track
q=News+From+Nowhere&type=track
q=Nothing%27s+Gonna+Change+My+Love+For+You&type=track
q=Now+And+Forever&type=track
q=O+Come+All+Ye+Faithful

q=Who+Can+Blame+You&type=track
q=Will+You+Be+Leaving&type=track
q=Your+Long+Journey&type=track
q=Ain%27t+No+Good+To+Cry&type=track
q=Bad+Rain&type=track
q=Been+Gone+Too+Long&type=track
q=Brothers+Of+The+Road&type=track
q=Demons&type=track
q=Every+Hungry+Woman&type=track
q=Good+Time+Feelin&type=track
q=Leavin%27&type=track
q=Let+Me+Ride&type=track
q=Mean+Old+World&type=track
q=Mean+Woman+Blues&type=track
q=Midnight+Blues&type=track
q=Morning+Dew&type=track
q=Mystery+Woman&type=track
q=Nancy&type=track
q=Need+Your+Love+So+Bad&type=track
q=Never+Knew+How+Much&type=track
q=No+Money+Down&type=track
q=Old+Friend&type=track
q=Sail+Away&type=track
q=Sailin%27+%27cross+The+Devil%27s+Sea&type=track
q=Shine+It+On&type=track
q=Straight+From+The+Heart&type=track
q=The+Heat+Is+On&type=track
q=The+Weight&type=track
q=Things+You+Used+To+Do&type=track
q=Two+Rights&type=track
q=Woman+Across+The+River&type=track
q=You+Don%27t+Love+Me&type=track
q=Afternoons+In+Utopia&type=track
q=Ain%27t+It+Strange&type=

q=Jesus%27+Brother+Bob&type=track
q=Kill+The+Dog+Next+Door&type=track
q=Login+To+You&type=track
q=Malcolm&type=track
q=Me+Like+Hockey%21&type=track
q=Mounted+Animal+Nature+Trail&type=track
q=My+Voice+Is+Changing&type=track
q=New+Car+Smell&type=track
q=Oh+God%2C+I%27m+Santa+Claus&type=track
q=Particle+Board&type=track
q=Proud+T+O+Be+A+Banker&type=track
q=Proud+To+Be+Canadian&type=track
q=Really+Scary&type=track
q=Steel+Drivin%27+Man&type=track
q=The+Ballad+Of+Dan&type=track
q=The+Golf+Song&type=track
q=The+Happy+Happy+Birthday+Song&type=track
q=The+Monkey+Song&type=track
q=The+Mountie+Song&type=track
q=The+War+Of+1812&type=track
q=Tv+Weather+Guy&type=track
q=17&type=track
q=All+The+Small+Things+%28Blink+182+Cover%29&type=track
q=Anything+But+Ordinary&type=track
q=Bad+Reputation&type=track
q=Complicated&type=track
q=Contagious&type=track
q=Despise+You&type=track
q=Destiny&type=track
q=Eternity&type=track
q=Everyday+Again&type=track
q=Fall+Into+You&type=track
q=Fall+To+Pieces&type=track
q

q=California+Dremaing&type=track
q=Car+Crazy+Cutie&type=track
q=Caroline%2C+No&type=track
q=Cherry%2C+Cherry+Coupe&type=track
q=Christmas+Day&type=track
q=Christmas+Time+Is+Here+Again&type=track
q=Crack+At+Your+Love&type=track
q=Cuddle+Up&type=track
q=Do+You+Wanna+Dance%3F&type=track
q=Endless+Harmony&type=track
q=Frosty+The+Snowman&type=track
q=Funky+Pretty&type=track
q=Girls+On+The+Beach&type=track
q=Good+To+My+Baby&type=track
q=Good+Vibrations&type=track
q=Graduation+Day&type=track
q=Help+Me%2C+Rhonda&type=track
q=Here+She+Comes&type=track
q=Here+Today&type=track
q=Hey+Little+Tomboy&type=track
q=Hold+On+Dear+Brother&type=track
q=Hot+Fun+In+The+Summertime&type=track
q=How+She+Boogalooed+It&type=track
q=Hushabye&type=track
q=I+Can+Hear+Music&type=track
q=I+Do&type=track
q=I+Do+Love+You&type=track
q=I+Get+Around&type=track
q=I+Just+Got+My+Pay&type=track
q=I+Should+Have+Known+Better&type=track
q=I%27m+The+Pied+Piper&type=track
q=I%27m+Waiting+For+The+Day&type=track
q=In+My+Car&type=trac

...

q=Lady%27s+Man&type=track
q=Lazy+Bones&type=track
q=Let%27s+Pretend&type=track
q=Letter+Edged+In+Black&type=track
q=Like+A+Bird&type=track
q=Lili+Marlene&type=track
q=Linda+Lou&type=track
q=Little+Britches&type=track
q=Little+Buddy&type=track
q=Little+Children&type=track
q=Little+Stranger+%28In+A+Manger%29&type=track
q=Lonesome+7-7203&type=track
q=Mama+Tried&type=track
q=Mansion+On+The+Hill&type=track
q=Marriage+And+Divorce&type=track
q=Marriage+Vow&type=track
q=Moanin%27&type=track
q=30+Pieces+Of+Silver&type=track
q=A+Home+In+Heaven&type=track
q=A+Mansion+On+The+Hill&type=track
q=A+Picture+From+Life%27s+Other+Side&type=track
q=A+Stranger+In+The+Night&type=track
q=A+Teardrop+On+A+Rose&type=track
q=All+The+Love+I+Ever+Had&type=track
q=Angel+Of+Death&type=track
q=Are+You+Building+A+Temple+In+Heaven&type=track
q=Baby+We%27re+Really+In+Love&type=track
q=Bayou+Pon+Pon&type=track
q=Cajun+Baby&type=track
q=California+Zephyr&type=track
q=Cold+Cold+Heart&type=track
q=Cool+Water&type=track
q=Dea

q=Shadow+Face&type=track
q=She+Went+A+Little+Bit+Farther&type=track
q=Something+To+Believe+In&type=track
q=Standing+In+The+Shadows&type=track
q=Stirrin%27+It+Up&type=track
q=Take+These+Chains+From+My+Heart&type=track
q=Tennessee+Stud&type=track
q=That+Ain%27t+Good&type=track
q=That%27s+How+I+Wanted+It+To+Be&type=track
q=That%27s+How+They+Do+It+In+Dixie&type=track
q=The+Blues+Man&type=track
q=The+Chair+That+Rocked+Us+All&type=track
q=The+Coalition+To+Ban+Coalitions&type=track
q=The+Count+Song&type=track
q=The+Kind+Of+Woman+I+Got&type=track
q=The+Last+Love+Song&type=track
q=The+Nashville+Scene&type=track
q=The+Pressure+Is+On&type=track
q=The+Ride&type=track
q=The+Same+Old+Story&type=track
q=A+Minute+Without+You&type=track
q=Ain%27t+No+Sunshine&type=track
q=Being+Me&type=track
q=Believe&type=track
q=Best+Days+Of+My+Life&type=track
q=Blue+Sky&type=track
q=Boomerang&type=track
q=Breaktown&type=track
q=Bridges+Of+Stone&type=track
q=Call+Me&type=track
q=Can%27t+Always+Get+What+You+Want&type=t

q=Cruel+Nights&type=track
q=Dear+Old+America&type=track
q=Desire&type=track
q=Dog+And+Butterfly&type=track
q=Down+On+Me&type=track
q=Dreamboat+Annie&type=track
q=Enough&type=track
q=Even+It+Up&type=track
q=Heartless&type=track
q=Hijinx&type=track
q=How+Can+I+Refuse&type=track
q=I+Want+You+So+Bad&type=track
q=I+Want+Your+World+To+Turn&type=track
q=If+Looks+Could+Kill&type=track
q=I%27ll+Be+Your+Song&type=track
q=I%27ll+Never+Stop+Loving+You&type=track
q=I%27ve+Got+The+Music+In+Me&type=track
q=Jealously&type=track
q=Kick+It+Out&type=track
q=Language+Of+Love&type=track
q=Lighter+Touch&type=track
q=Little+Queen&type=track
q=Lost+Angel&type=track
q=Love+Alive&type=track
q=Love+Mistake&type=track
q=Magic+Man&type=track
q=Mary&type=track
q=Million+Miles&type=track
q=Mistral+Wind&type=track
q=Mona+Lisas+And+Mad+Hatters&type=track
q=Move+On&type=track
q=My+Crazy+Head&type=track
q=Nada+One&type=track
q=One+Word&type=track
q=Pennsylvania&type=track
q=Pilot&type=track
q=Private+Audition&type=track

q=Bless+The+Lord&type=track
q=By+Your+Side&type=track
q=Can%27t+Stop+Praising&type=track
q=Can%27t+Stop+Talking&type=track
q=Centre+Of+My+Life&type=track
q=Children+Of+The+Light&type=track
q=Church+On+Fire&type=track
q=Closer+Than+You+Know&type=track
q=Consuming+Fire&type=track
q=Cry+Of+The+Broken&type=track
q=Deeper&type=track
q=Desert+Song&type=track
q=Even+When+It+Hurts+%28Praise+Song%29&type=track
q=Ever+Living+God&type=track
q=Every+Time&type=track
q=Everyday&type=track
q=Everything+That+Has+Breath&type=track
q=Everything+To+Me&type=track
q=Faith&type=track
q=Faithful&type=track
q=Father%2C+I&type=track
q=Father+Of+Creation&type=track
q=Fire+Fall+Down&type=track
q=For+This+Cause&type=track
q=Forever&type=track
q=Forever+And+A+Day&type=track
q=Found&type=track
q=From+God+Above&type=track
q=Glory+To+The+King&type=track
q=Go&type=track
q=God+Is+Able&type=track
q=God+Is+Great&type=track
q=God+Is+In+The+House&type=track
q=Gonna+Be+Alright&type=track
q=Grace+Abounds&type=track
q=Hallelu

ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=Just+One+Look&type=track (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fd42ce98ed0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [27]:
df = pd.DataFrame({'title': title, 'date': dates, 'artist': artist})
df

,title,date,artist
0,"Andante, Andante",1980,ABBA
1,"Andante, Andante",1993,ABBA
2,As Good As New,1979,ABBA
3,As Good As New,2014-01-01,ABBA
4,Bang-A-Boomerang,1975,ABBA
...,...,...,...
42508,Venus Doom,2007-09-07,HIM
42509,Wicked Game,2012-11-06,HIM
42510,You Are The One,2014-12-15,HIM
42511,Your Sweet 666,2014-12-15,HIM


In [43]:
df = pd.DataFrame({'title': title, 'date': dates, 'artist': artist})

In [29]:
df.to_csv('songs_plus_dates')

In [55]:
import pandas as pd
df = pd.read_csv("songs_plus_dates")

In [59]:
for i, row in df.iterrows():
    df.at[i, "date"] = row.date[:4]
df

,Unnamed: 0,title,date,artist
0,0,"Andante, Andante",1980,ABBA
1,1,"Andante, Andante",1993,ABBA
2,2,As Good As New,1979,ABBA
3,3,As Good As New,2014,ABBA
4,4,Bang-A-Boomerang,1975,ABBA
...,...,...,...,...
42508,42508,Venus Doom,2007,HIM
42509,42509,Wicked Game,2012,HIM
42510,42510,You Are The One,2014,HIM
42511,42511,Your Sweet 666,2014,HIM


In [65]:
df.columns = ['song', 'date', 'artist']
df

,song,date,artist
0,"Andante, Andante",1980,ABBA
1,"Andante, Andante",1993,ABBA
2,As Good As New,1979,ABBA
3,As Good As New,2014,ABBA
4,Bang-A-Boomerang,1975,ABBA
...,...,...,...
42508,Venus Doom,2007,HIM
42509,Wicked Game,2012,HIM
42510,You Are The One,2014,HIM
42511,Your Sweet 666,2014,HIM


In [61]:
df4 = pd.read_csv("../songdata.csv")
df4

,Unnamed: 0,artist,song,link,text,Genre1,Genre2
0,0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...",Pop,Rock
1,1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",Pop,Rock
2,2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,Pop,Rock
3,3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,Pop,Rock
4,4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Pop,Rock
...,...,...,...,...,...,...,...
57645,57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...,Reggae,Pop
57646,57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...,Reggae,Pop
57647,57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...,Electronic,Pop
57648,57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...,Electronic,Pop


In [81]:
import numpy as np

In [99]:
songs_dates = pd.merge(df, df4, on=["song", "artist"])
songs_dates

,song,date,artist,Unnamed: 0,link,text,Genre1,Genre2
0,"Andante, Andante",1980,ABBA,1,/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",Pop,Rock
1,"Andante, Andante",1993,ABBA,1,/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",Pop,Rock
2,As Good As New,1979,ABBA,2,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,Pop,Rock
3,As Good As New,2014,ABBA,2,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,Pop,Rock
4,Bang-A-Boomerang,1975,ABBA,4,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Pop,Rock
...,...,...,...,...,...,...,...,...
42508,Venus Doom,2007,HIM,36047,/h/him/venus+doom_20447343.html,Leave all behind now to watch her crawl \nThr...,Rock,Pop
42509,Wicked Game,2012,HIM,36049,/h/him/wicked+game_20063721.html,The world was on fire \nNo one could save me ...,Rock,Pop
42510,You Are The One,2014,HIM,36050,/h/him/you+are+the+one_10139940.html,No I won't surrender \nAt any cost \nYou're ...,Rock,Pop
42511,Your Sweet 666,2014,HIM,36051,/h/him/your+sweet+666_10139928.html,There are things you should know \nThe distan...,Rock,Pop


In [100]:
songs_dates.drop_duplicates(subset="song", keep='first', inplace=True)
songs_dates

,song,date,artist,Unnamed: 0,link,text,Genre1,Genre2
0,"Andante, Andante",1980,ABBA,1,/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",Pop,Rock
2,As Good As New,1979,ABBA,2,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,Pop,Rock
4,Bang-A-Boomerang,1975,ABBA,4,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Pop,Rock
7,Cassandra,1981,ABBA,6,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...,Pop,Rock
8,Chiquitita,1979,ABBA,7,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc...",Pop,Rock
...,...,...,...,...,...,...,...,...
42497,Vampire Heart,2005,HIM,36046,/h/him/vampire+heart_20436983.html,You can't escape the wrath of my heart \nBeat...,Rock,Pop
42498,Venus Doom,2007,HIM,36047,/h/him/venus+doom_20447343.html,Leave all behind now to watch her crawl \nThr...,Rock,Pop
42509,Wicked Game,2012,HIM,36049,/h/him/wicked+game_20063721.html,The world was on fire \nNo one could save me ...,Rock,Pop
42510,You Are The One,2014,HIM,36050,/h/him/you+are+the+one_10139940.html,No I won't surrender \nAt any cost \nYou're ...,Rock,Pop


In [102]:
songs_dates = songs_dates[songs_dates.Genre1.astype(str) != "nan"]

In [103]:
genres = np.unique(songs_dates.Genre2)

In [104]:
songs_dates['Genre1'].value_counts()

Rock                      10080
Pop                        2149
Folk, World, & Country     1630
Electronic                  765
Jazz                        538
Funk / Soul                 413
Hip Hop                     270
Reggae                       90
Stage & Screen               17
Name: Genre1, dtype: int64

In [105]:
len(np.unique(songs_dates.artist))

362

In [106]:
songs_dates.to_csv("songs_plus_dates.csv")